In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [7]:
from langchain_core.tools import tool
import os
import requests
from dotenv import load_dotenv

load_dotenv()

@tool
def get_places_json(city: str, query: str = "attractions") -> list:
    """
    Fetches top places (e.g., attractions, restaurants) in a city using the Foursquare Places API
    and returns them as a structured JSON list.

    Parameters:
        city (str): Name of the city (e.g., 'Kolkata')
        query (str): Type of places to search for (e.g., 'attractions', 'museums')

    Returns:
        list: A list of dictionaries, each containing:
            - name: Place name
            - categories: List of category names
            - address: Formatted address
            - phone: Telephone number if available
            - website: Website URL if available
"""
    api_key = os.getenv("FOURSQUARE_API_KEY")
    if not api_key:
        return [{"error": "Missing FOURSQUARE_API_KEY"}]

    url = "https://places-api.foursquare.com/places/search"
    headers = {
        "accept": "application/json",
        "X-Places-Api-Version": "2025-06-17",
        "authorization": api_key
    }
    params = {
        "near": city,
        "query": query,
        "limit": 5
    }

    response = requests.get(url, headers=headers, params=params)
    if response.status_code != 200:
        return [{"error": f"Foursquare API error: {response.text}"}]

    results = response.json().get("results", [])
    if not results:
        return [{"message": f"No results found for '{query}' in {city}."}]

    extracted = []
    for place in results:
        name = place.get("name", "Unknown")
        categories = [cat.get("name") for cat in place.get("categories", [])]
        address = place['location']['formatted_address']
        phone = place.get("tel", None)
        website = place.get("website", None)

        extracted.append({
            "name": name,
            "categories": categories,
            "address": address,
            "phone": phone,
            "website": website
        })

    return extracted

In [32]:
get_places_json.description

"Fetches top places (e.g., attractions, restaurants) in a city using the Foursquare Places API\nand returns them as a structured JSON list.\n\nParameters:\n    city (str): Name of the city (e.g., 'Kolkata')\n    query (str): Type of places to search for (e.g., 'attractions', 'museums')\n\nReturns:\n    list: A list of dictionaries, each containing:\n        - name: Place name\n        - categories: List of category names\n        - address: Formatted address\n        - phone: Telephone number if available\n        - website: Website URL if available"

In [33]:
get_places_json.args

{'city': {'title': 'City', 'type': 'string'},
 'query': {'default': 'attractions', 'title': 'Query', 'type': 'string'}}

In [41]:
get_places_json.invoke(
    {
        'city': 'south kolkata',
        'query': 'hotels'
    }
)

[{'name': 'The Oberoi Grand',
  'categories': ['Hotel'],
  'address': '15, Jawaharlal Nehru Rd, Kolkata 700013, West Bengal',
  'phone': '033 2249 2323',
  'website': 'http://www.oberoihotels.com'},
 {'name': 'Hyatt Regency Kolkata',
  'categories': ['Hotel'],
  'address': 'Ja-1 Sector Iii Salt Lake City, Kolkata 700098, West Bengal',
  'phone': '033 6820 1234',
  'website': 'https://www.hyatt.com/en-US/hotel/india/hyatt-regency-kolkata/kolka'},
 {'name': 'Lalit Great Eastern Hotel',
  'categories': ['Hotel'],
  'address': 'Hemanta Basu Sarani, Kolkata 700001, West Bengal',
  'phone': '033 4444 7777',
  'website': None},
 {'name': 'Taj Bengal',
  'categories': ['Hotel', 'Hotel Bar'],
  'address': '34B, Belvedere Rd, Kolkata 700027, West Bengal',
  'phone': '033 2223 3939',
  'website': 'https://taj.tajhotels.com/en-in/taj-bengal-kolkata/?gclid=CLzEtriEvM0CFdMPaAoddJQLhQ'},
 {'name': 'ITC Sonar',
  'categories': ['Hotel'],
  'address': 'Jbs Haldane Avenue, Kolkata 700046, West Bengal',


In [18]:
from langchain_core.tools import tool
import os
import requests
from dotenv import load_dotenv
from datetime import datetime, timedelta

load_dotenv()

@tool
def get_hotels_by_city(city: str) -> list:
    """
    Get top hotels with prices per night in INR for the given city using Hotels4 (RapidAPI).
    
    Parameters:
        city (str): City name (e.g., "Kolkata")

    Returns:
        list: List of hotels with name, price, and address
    """

    headers = {
        "x-rapidapi-key": os.getenv("RAPIDAPI_KEY"),
        "x-rapidapi-host": "hotels4.p.rapidapi.com"
    }

    # Step 1: Get gaiaId (destination identifier)
    location_url = "https://hotels4.p.rapidapi.com/locations/v3/search"
    location_params = {
        "q": city,
        "locale": "en_US",
        "langid": "1033",
        "siteid": "3000000"
    }

    location_resp = requests.get(location_url, headers=headers, params=location_params)
    if location_resp.status_code != 200:
        return [{"error": f"Location fetch failed: {location_resp.text}"}]

    try:
        gaia_id = location_resp.json()["sr"][0]["gaiaId"]
    except Exception:
        return [{"error": f"Could not find destination ID for city: {city}"}]

    # Step 2: Get hotel list for destination
    hotel_url = "https://hotels4.p.rapidapi.com/properties/v2/list"
    today = datetime.today()
    checkin_date = today.strftime("%Y-%m-%d")
    checkout_date = (today + timedelta(days=1)).strftime("%Y-%m-%d")

    payload = {
        "currency": "INR",
        "locale": "en_US",
        "siteId": 3000000,
        "destination": {"regionId": gaia_id},
        "checkInDate": {
            "day": today.day,
            "month": today.month,
            "year": today.year
        },
        "checkOutDate": {
            "day": (today + timedelta(days=1)).day,
            "month": (today + timedelta(days=1)).month,
            "year": (today + timedelta(days=1)).year
        },
        "rooms": [{"adults": 1}],
        "resultsStartingIndex": 0,
        "resultsSize": 25,
        # "sort": "PRICE_LOW_TO_HIGH",
        "filters": {}
    }

    hotel_resp = requests.post(hotel_url, json=payload, headers=headers)
    if hotel_resp.status_code != 200:
        return [{"error": f"Hotel list fetch failed: {hotel_resp.text}"}]

    hotels_raw = hotel_resp.json().get("data", {}).get("propertySearch", {}).get("properties", [])
    if not hotels_raw:
        return [{"message": f"No hotel results found in {city}"}]

    results = []
    for hotel in hotels_raw:
        print(hotel)
        print('*'*50)
        print()
        name = hotel.get("name")
        address = hotel.get("address", {}).get("addressLine", "No address provided")
        price = hotel.get("price", {}).get("lead", {}).get("formatted", "N/A")
        results.append({
            "name": name,
            "address": address,
            "price_per_night": price
        })

    return results

In [19]:
get_hotels_by_city.invoke({"city": "Kolkata"})

{'__typename': 'Property', 'id': '109413882', 'featuredMessages': [], 'name': 'Holiday Inn Express Kolkata New Town by IHG', 'availability': {'__typename': 'PropertyAvailability', 'available': True, 'minRoomsLeft': None}, 'propertyImage': {'__typename': 'PropertyImage', 'alt': '', 'fallbackImage': None, 'image': {'__typename': 'Image', 'description': 'Exterior', 'url': 'https://images.trvl-media.com/lodging/110000000/109420000/109413900/109413882/w6048h4036x2y0-abd9b79e.jpg?impolicy=resizecrop&rw=455&ra=fit'}, 'subjectId': 91024}, 'destinationInfo': {'__typename': 'PropertyDestinationInfo', 'distanceFromDestination': {'__typename': 'Distance', 'unit': 'KILOMETER', 'value': 15.58}, 'distanceFromMessaging': '15.58 km from ISKCON Temple Siliguri', 'regionId': '553248635939586249'}, 'legalDisclaimer': None, 'listingFooter': None, 'mapMarker': {'__typename': 'MapMarker', 'label': '$42', 'latLong': {'__typename': 'Coordinates', 'latitude': 22.578858, 'longitude': 88.462298}}, 'neighborhood':

[{'name': 'Holiday Inn Express Kolkata New Town by IHG',
  'address': 'No address provided',
  'price_per_night': '$42'},
 {'name': 'ITC Royal Bengal, a Luxury Collection Hotel, Kolkata',
  'address': 'No address provided',
  'price_per_night': '$124'},
 {'name': 'ITC Sonar, a Luxury Collection Hotel, Kolkata',
  'address': 'No address provided',
  'price_per_night': '$110'},
 {'name': 'LAKESIDE SUITES',
  'address': 'No address provided',
  'price_per_night': '$31'},
 {'name': 'Royal Guest House',
  'address': 'No address provided',
  'price_per_night': '$14'},
 {'name': 'Fabexpress Golden Imperial',
  'address': 'No address provided',
  'price_per_night': '$20'},
 {'name': 'Southern Plaza',
  'address': 'No address provided',
  'price_per_night': '$59'},
 {'name': "OYO 15617 Abi's Inn",
  'address': 'No address provided',
  'price_per_night': '$17'},
 {'name': 'Fabexpress Amar Raj Villa',
  'address': 'No address provided',
  'price_per_night': '$13'},
 {'name': 'Fabhotel The Hasting

In [8]:
## currency exchange tool

@tool
def convert_currency(amount: float, to_currency: str, base: str = "USD") -> float:
    """
    Convert `amount` from `base` currency to `to_currency` using ExchangeRate-API open endpoint.
    """
    url = f"https://open.er-api.com/v6/latest/{base}"
    resp = requests.get(url)
    data = resp.json()
    rate = data["rates"].get(to_currency)
    if not rate:
        return {"error": f"Rate unavailable for {to_currency}"}
    return round(amount * rate, 2)

In [9]:
convert_currency.args

{'amount': {'title': 'Amount', 'type': 'number'},
 'to_currency': {'title': 'To Currency', 'type': 'string'},
 'base': {'default': 'USD', 'title': 'Base', 'type': 'string'}}

In [10]:
convert_currency.invoke(
    {
        'amount': 1,
        'to_currency': 'INR'
    }
)

85.55

In [2]:
from langchain_core.tools import tool
from dotenv import load_dotenv
import requests
import os

load_dotenv()

@tool
def get_flight_fares(from_code: str, to_code: str, date: str, adult: int = 1, type_: str = "economy") -> list:
    """
    Fetches flight fare data using the Flight Fare Search API on RapidAPI.

    Args:
        from_code (str): IATA code of departure airport (e.g., 'BLR')
        to_code (str): IATA code of arrival airport (e.g., 'CCU')
        date (str): Travel date in YYYY-MM-DD
        adult (int): Number of adult passengers (default: 1)
        type_ (str): Cabin class (default: 'economy')

    Returns:
        list: List of flights with key details: timing, pricing, stops, countries, and cabin info.
    """
    url = "https://flight-fare-search.p.rapidapi.com/v2/flights/"

    querystring = {
        "from": from_code,
        "to": to_code,
        "date": date,
        "adult": str(adult),
        "type": type_,
        "currency": "USD"
    }

    headers = {
        "x-rapidapi-key": os.getenv("RAPIDAPI_KEY"),
        "x-rapidapi-host": "flight-fare-search.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=querystring)
    print("🔍 Raw API response:", response.status_code, response.text)

    try:
        raw = response.json()
        flights = raw.get("results", [])
        if not isinstance(flights, list) or not flights:
            return [{"message": "No flights found."}]

        results = []
        for f in flights:
            stop_info = []
            stop_summary = f.get("stopSummary", {})

            # Extract intermediate stops if present
            if isinstance(stop_summary, dict):
                for key, val in stop_summary.items():
                    if key != "connectingTime" and isinstance(val, dict):
                        stop_info.append({
                            "intermediate_airport": val.get("airport", "Unknown"),
                            "stop_duration_minutes": val.get("stopDuration")
                        })

            results.append({
                "flight_code": f.get("flight_code"),
                "airline": f.get("flight_name"),
                "cabin_type": f.get("cabinType", "Unknown"),
                "stops": f.get("stops", "Unknown"),
                "departure_city": f.get("departureAirport", {}).get("city"),
                "departure_country": f.get("departureAirport", {}).get("country", {}).get("label"),
                "departure_time": f.get("departureAirport", {}).get("time"),
                "arrival_city": f.get("arrivalAirport", {}).get("city"),
                "arrival_country": f.get("arrivalAirport", {}).get("country", {}).get("label"),
                "arrival_time": f.get("arrivalAirport", {}).get("time"),
                "duration": f.get("duration", {}).get("text"),
                "price": f.get("totals", {}).get("total"),
                "currency": f.get("totals", {}).get("currency"),
                "intermediate_stops": stop_info if stop_info else None
            })

        return results

    except Exception as e:
        return [{"error": str(e)}]

In [3]:
get_flight_fares.description

"Fetches flight fare data using the Flight Fare Search API on RapidAPI.\n\nArgs:\n    from_code (str): IATA code of departure airport (e.g., 'BLR')\n    to_code (str): IATA code of arrival airport (e.g., 'CCU')\n    date (str): Travel date in YYYY-MM-DD\n    adult (int): Number of adult passengers (default: 1)\n    type_ (str): Cabin class (default: 'economy')\n\nReturns:\n    list: List of flights with key details: timing, pricing, stops, countries, and cabin info."

In [4]:
get_flight_fares.args

{'from_code': {'title': 'From Code', 'type': 'string'},
 'to_code': {'title': 'To Code', 'type': 'string'},
 'date': {'title': 'Date', 'type': 'string'},
 'adult': {'default': 1, 'title': 'Adult', 'type': 'integer'},
 'type_': {'default': 'economy', 'title': 'Type', 'type': 'string'}}

In [6]:
response = get_flight_fares.invoke({
    "from_code": "BLR",
    "to_code": "CCU",
    "date": "2025-07-01"
})

print(response)

🔍 Raw API response: 200 {"status":200,"searchData":{"from":"BLR","to":"CCU","date":"2025-07-01","type":"Economy","adult":1,"child":0,"infant":0,"currency":"USD"},"results":[{"id":"100393a9b64b249a848687aa774ecdaab34b38af411680e950605361f18bacf8","careerCode":"AI","flight_code":"AI-2824","flight_name":"Air India","stops":"1 Stop","cabinType":"Economy","baggage":{"cabin":{"qty":1,"unit":"KG","text":"7 KG","allowance":7},"checkIn":{"allowance":15,"qty":"W","unit":"K","text":"15 KG","refNumber":1,"quantity":1,"totalWeight":15}},"currency":"USD","departureAirport":{"time":"2025-07-01T14:15:00","code":"BLR","tz":"Asia/Kolkata","timeZone":"5.50","type":"2","label":"Bengaluru International Airport","country":{"label":"India","code":"IN"},"city":"Bangalore"},"arrivalAirport":{"time":"2025-07-02T00:30:00","code":"CCU","tz":"Asia/Kolkata","timeZone":"5.50","type":"2","label":"Netaji Subhas Chandra Bose Airport","country":{"label":"India","code":"IN"},"city":"Kolkata"},"path":["AI-2824","AI-9783"]

In [7]:
response

[{'flight_code': 'AI-2824',
  'airline': 'Air India',
  'cabin_type': 'Economy',
  'stops': '1 Stop',
  'departure_city': 'Bangalore',
  'departure_country': 'India',
  'departure_time': '2025-07-01T14:15:00',
  'arrival_city': 'Kolkata',
  'arrival_country': 'India',
  'arrival_time': '2025-07-02T00:30:00',
  'duration': '10h 15m',
  'price': 97.92,
  'currency': 'USD',
  'intermediate_stops': [{'intermediate_airport': 'MAA',
    'stop_duration_minutes': 390}]},
 {'flight_code': 'AI-2824',
  'airline': 'Air India',
  'cabin_type': 'Economy',
  'stops': '1 Stop',
  'departure_city': 'Bangalore',
  'departure_country': 'India',
  'departure_time': '2025-07-01T14:15:00',
  'arrival_city': 'Kolkata',
  'arrival_country': 'India',
  'arrival_time': '2025-07-02T12:20:00',
  'duration': '22h 05m',
  'price': 97.92,
  'currency': 'USD',
  'intermediate_stops': [{'intermediate_airport': 'MAA',
    'stop_duration_minutes': 1105}]},
 {'flight_code': 'AI-9566',
  'airline': 'Air India',
  'cabin_

{'id': '598773c820cf2ad796f1771aa043f6f019f262ac6b148bf6a725673a57a10086', 'careerCode': 'IX', 'flight_code': 'IX-5610', 'flight_name': 'Air India Express', 'stops': '1 Stop', 'cabinType': 'Economy', 'baggage': {'cabin': {'allowance': 7, 'qty': 1, 'unit': 'KG', 'text': '7 KG'}, 'checkIn': {'allowance': 15, 'qty': 1, 'unit': 'KG', 'text': '15 KG', 'totalWeight': 15, 'quantity': 1}, 'baggageOptionsAvailable': False}, 'currency': 'USD', 'departureAirport': {'time': '2025-07-01T05:00:00', 'code': 'BLR', 'tz': 'Asia/Kolkata', 'timeZone': '5.50', 'type': '2', 'label': 'Bengaluru International Airport', 'country': {'label': 'India', 'code': 'IN'}, 'city': 'Bangalore'}, 'arrivalAirport': {'time': '2025-07-01T12:20:00', 'code': 'CCU', 'tz': 'Asia/Kolkata', 'timeZone': '5.50', 'type': '2', 'label': 'Netaji Subhas Chandra Bose Airport', 'country': {'label': 'India', 'code': 'IN'}, 'city': 'Kolkata'}, 'path': ['IX-5610', 'IX-1256'], 'duration': {'text': '07h 20m', 'value': 440}, 'stopSummary': {'0': {'airport': 'MAA', 'stopDuration': 215}, 'connectingTime': None}, 'totals': {'currency': 'USD', 'baggage': None, 'penalty': None, 'total': 55.207840000000004, 'tax': None, 'base': 55.207840000000004}}


{'id': 'd8f07b3f1c1c0d13445db7a76a89a07ba0eade2b70ab4f513a2b528e71df846f', 'careerCode': 'IX', 'flight_code': 'IX-5012', 'flight_name': 'Air India Express', 'stops': '1 Stop', 'cabinType': 'Economy', 'baggage': {'cabin': {'allowance': 7, 'qty': 1, 'unit': 'KG', 'text': '7 KG'}, 'checkIn': {'allowance': 15, 'qty': 1, 'unit': 'KG', 'text': '15 KG', 'totalWeight': 15, 'quantity': 1}, 'baggageOptionsAvailable': False}, 'currency': 'USD', 'departureAirport': {'time': '2025-07-01T05:15:00', 'code': 'BLR', 'tz': 'Asia/Kolkata', 'timeZone': '5.50', 'type': '2', 'label': 'Bengaluru International Airport', 'country': {'label': 'India', 'code': 'IN'}, 'city': 'Bangalore'}, 'arrivalAirport': {'time': '2025-07-01T18:30:00', 'code': 'CCU', 'tz': 'Asia/Kolkata', 'timeZone': '5.50', 'type': '2', 'label': 'Netaji Subhas Chandra Bose Airport', 'country': {'label': 'India', 'code': 'IN'}, 'city': 'Kolkata'}, 'path': ['IX-5012', 'IX-1224'], 'duration': {'text': '13h 15m', 'value': 795}, 'stopSummary': {'0': {'airport': 'HYD', 'stopDuration': 330}, 'connectingTime': None}, 'totals': {'currency': 'USD', 'baggage': None, 'penalty': None, 'total': 61.833760000000005, 'tax': None, 'base': 61.833760000000005}}


{'id': '74afe26b11cfaf84ebbc4166f3d706e270f311194100e36d474fe2a3506cb3cc', 'careerCode': 'IX', 'flight_code': 'IX-5012', 'flight_name': 'Air India Express', 'stops': '1 Stop', 'cabinType': 'Economy', 'baggage': {'cabin': {'allowance': 7, 'qty': 1, 'unit': 'KG', 'text': '7 KG'}, 'checkIn': {'allowance': 15, 'qty': 1, 'unit': 'KG', 'text': '15 KG', 'totalWeight': 15, 'quantity': 1}, 'baggageOptionsAvailable': False}, 'currency': 'USD', 'departureAirport': {'time': '2025-07-01T05:15:00', 'code': 'BLR', 'tz': 'Asia/Kolkata', 'timeZone': '5.50', 'type': '2', 'label': 'Bengaluru International Airport', 'country': {'label': 'India', 'code': 'IN'}, 'city': 'Bangalore'}, 'arrivalAirport': {'time': '2025-07-01T20:25:00', 'code': 'CCU', 'tz': 'Asia/Kolkata', 'timeZone': '5.50', 'type': '2', 'label': 'Netaji Subhas Chandra Bose Airport', 'country': {'label': 'India', 'code': 'IN'}, 'city': 'Kolkata'}, 'path': ['IX-5012', 'IX-2825'], 'duration': {'text': '15h 10m', 'value': 910}, 'stopSummary': {'0': {'airport': 'HYD', 'stopDuration': 445}, 'connectingTime': None}, 'totals': {'currency': 'USD', 'baggage': None, 'penalty': None, 'total': 61.833760000000005, 'tax': None, 'base': 61.833760000000005}}


{'id': 'f1ba1853c96aba6b6367411e6152c0f663a902d8a8fcb1226a0263cd5f674236', 'careerCode': 'IX', 'flight_code': 'IX-2870', 'flight_name': 'Air India Express', 'stops': '1 Stop', 'cabinType': 'Economy', 'baggage': {'cabin': {'allowance': 7, 'qty': 1, 'unit': 'KG', 'text': '7 KG'}, 'checkIn': {'allowance': 15, 'qty': 1, 'unit': 'KG', 'text': '15 KG', 'totalWeight': 15, 'quantity': 1}, 'baggageOptionsAvailable': False}, 'currency': 'USD', 'departureAirport': {'time': '2025-07-01T07:35:00', 'code': 'BLR', 'tz': 'Asia/Kolkata', 'timeZone': '5.50', 'type': '2', 'label': 'Bengaluru International Airport', 'country': {'label': 'India', 'code': 'IN'}, 'city': 'Bangalore'}, 'arrivalAirport': {'time': '2025-07-01T18:30:00', 'code': 'CCU', 'tz': 'Asia/Kolkata', 'timeZone': '5.50', 'type': '2', 'label': 'Netaji Subhas Chandra Bose Airport', 'country': {'label': 'India', 'code': 'IN'}, 'city': 'Kolkata'}, 'path': ['IX-2870', 'IX-1224'], 'duration': {'text': '10h 55m', 'value': 655}, 'stopSummary': {'0': {'airport': 'HYD', 'stopDuration': 330}, 'connectingTime': None}, 'totals': {'currency': 'USD', 'baggage': None, 'penalty': None, 'total': 61.833760000000005, 'tax': None, 'base': 61.833760000000005}}


{'id': '3cc0a45d6ed4ef6c8a72807c3cc9e9c0b5c5964cb930780c9f8785868460f7f6', 'careerCode': 'IX', 'flight_code': 'IX-1574', 'flight_name': 'Air India Express', 'stops': 'Direct', 'cabinType': 'Economy', 'baggage': {'cabin': {'allowance': 7, 'qty': 1, 'unit': 'KG', 'text': '7 KG'}, 'checkIn': {'allowance': 15, 'qty': 1, 'unit': 'KG', 'text': '15 KG', 'totalWeight': 15, 'quantity': 1}, 'baggageOptionsAvailable': False}, 'currency': 'USD', 'departureAirport': {'time': '2025-07-01T14:30:00', 'code': 'BLR', 'tz': 'Asia/Kolkata', 'timeZone': '5.50', 'type': '2', 'label': 'Bengaluru International Airport', 'country': {'label': 'India', 'code': 'IN'}, 'city': 'Bangalore'}, 'arrivalAirport': {'time': '2025-07-01T17:20:00', 'code': 'CCU', 'tz': 'Asia/Kolkata', 'timeZone': '5.50', 'type': '2', 'label': 'Netaji Subhas Chandra Bose Airport', 'country': {'label': 'India', 'code': 'IN'}, 'city': 'Kolkata'}, 'path': ['IX-1574'], 'duration': {'text': '02h 50m', 'value': 170}, 'stopSummary': {'connectingTime': None}, 'totals': {'currency': 'USD', 'baggage': None, 'penalty': None, 'total': 74.02752000000001, 'tax': None, 'base': 74.02752000000001}}


In [11]:
response = get_flight_fares.invoke({
    "from_code": "DEL",
    "to_code": "BOM",
    "date": "2025-07-02"
})

response

🔍 Raw API response: 200 {"status":200,"searchData":{"from":"DEL","to":"BOM","date":"2025-07-02","type":"Economy","adult":1,"child":0,"infant":0,"currency":"USD"},"results":[{"id":"fe884a4fe0b4c9f8d979b1b96d7ee484b0089140eaafab5693636290826ae616","careerCode":"SG","flight_code":"SG-8157","flight_name":"Spicejet","stops":"Direct","cabinType":"Economy","baggage":{"cabin":{"allowance":7,"qty":1,"unit":"KG","text":"7 KG"},"checkIn":{"allowance":15,"qty":1,"unit":"KG","text":"15 KG","totalWeight":15,"quantity":1},"baggageOptionsAvailable":false},"currency":"USD","departureAirport":{"time":"2025-07-02T06:00:00","code":"DEL","tz":"Asia/Kolkata","timeZone":"5.50","type":"1","label":"Indira Gandhi International Airport","country":{"label":"India","code":"IN"},"city":"New Delhi"},"arrivalAirport":{"time":"2025-07-02T08:25:00","code":"BOM","tz":"Asia/Kolkata","timeZone":"5.50","type":"1","label":"Chhatrapati Shivaji International Airport","country":{"label":"India","code":"IN"},"city":"Mumbai"},"p

[{'flight_code': 'SG-8157',
  'airline': 'Spicejet',
  'cabin_type': 'Economy',
  'stops': 'Direct',
  'departure_city': 'New Delhi',
  'departure_country': 'India',
  'departure_time': '2025-07-02T06:00:00',
  'arrival_city': 'Mumbai',
  'arrival_country': 'India',
  'arrival_time': '2025-07-02T08:25:00',
  'duration': '02h 25m',
  'price': 57.147200000000005,
  'currency': 'USD',
  'intermediate_stops': None},
 {'flight_code': 'SG-9248',
  'airline': 'Spicejet',
  'cabin_type': 'Economy',
  'stops': 'Direct',
  'departure_city': 'New Delhi',
  'departure_country': 'India',
  'departure_time': '2025-07-02T09:00:00',
  'arrival_city': 'Mumbai',
  'arrival_country': 'India',
  'arrival_time': '2025-07-02T11:30:00',
  'duration': '02h 30m',
  'price': 56.2224,
  'currency': 'USD',
  'intermediate_stops': None},
 {'flight_code': 'SG-9246',
  'airline': 'Spicejet',
  'cabin_type': 'Economy',
  'stops': 'Direct',
  'departure_city': 'New Delhi',
  'departure_country': 'India',
  'departure_

In [12]:
query = {
    "from_code": "BLR",
    "to_code": "BKK",
    "date": "2025-07-02"
}

response = get_flight_fares.invoke(query)
response

🔍 Raw API response: 200 {"status":200,"searchData":{"from":"BLR","to":"BKK","date":"2025-07-02","type":"Economy","adult":1,"child":0,"infant":0,"currency":"USD"},"results":[{"id":"d51b0f3c4a4c6c1f4c971d6af2a43836c668ee1d97c507b0fa2e7b11efa3a562","careerCode":"6E","flight_code":"6E-1055","flight_name":"Indigo","stops":"Direct","cabinType":"Economy","baggage":{"cabin":{"allowance":7,"qty":1,"unit":"KG","text":"7 KG"},"checkIn":{"allowance":20,"unit":"KG","qty":1,"text":"20 KG","totalWeight":20,"quantity":1},"baggageOptionsAvailable":false},"currency":"USD","departureAirport":{"time":"2025-07-02T05:25:00","code":"BLR","tz":"Asia/Kolkata","timeZone":"5.50","type":"2","label":"Bengaluru International Airport","country":{"label":"India","code":"IN"},"city":"Bangalore"},"arrivalAirport":{"time":"2025-07-02T10:55:00","code":"BKK","tz":"Asia/Bangkok","timeZone":"7.00","type":"1","label":"Suvarnabhumi Airport","country":{"label":"Thailand","code":"TH"},"city":"Bangkok"},"path":["6E-1055"],"durat

[{'flight_code': '6E-1055',
  'airline': 'Indigo',
  'cabin_type': 'Economy',
  'stops': 'Direct',
  'departure_city': 'Bangalore',
  'departure_country': 'India',
  'departure_time': '2025-07-02T05:25:00',
  'arrival_city': 'Bangkok',
  'arrival_country': 'Thailand',
  'arrival_time': '2025-07-02T10:55:00',
  'duration': '04h 00m',
  'price': 134.36800000000002,
  'currency': 'USD',
  'intermediate_stops': None},
 {'flight_code': '6E-5216',
  'airline': 'Indigo',
  'cabin_type': 'Economy',
  'stops': '1 Stop',
  'departure_city': 'Bangalore',
  'departure_country': 'India',
  'departure_time': '2025-07-02T08:50:00',
  'arrival_city': 'Bangkok',
  'arrival_country': 'Thailand',
  'arrival_time': '2025-07-02T22:40:00',
  'duration': '12h 20m',
  'price': 177.616,
  'currency': 'USD',
  'intermediate_stops': [{'intermediate_airport': 'BOM',
    'stop_duration_minutes': 185}]},
 {'flight_code': '6E-503',
  'airline': 'Indigo',
  'cabin_type': 'Economy',
  'stops': '1 Stop',
  'departure_c